# Looker Dashboard Interativo COVID-19

Caderno de Desenvolvimento <br>
Curadoria de [Rafael Barbosa](https://www.linkedin.com/in/barbosa89/)

---

<iframe width="600" height="900" src="https://lookerstudio.google.com/embed/reporting/f923059d-f701-4b2d-9a40-8b254d2f6350/page/96tjD" frameborder="0" style="border:0" allowfullscreen sandbox="allow-storage-access-by-user-activation allow-scripts allow-same-origin allow-popups allow-popups-to-escape-sandbox"></iframe>

## TLDR

 * ### **Dashboard Interativo**: [Looker Studio](https://lookerstudio.google.com/reporting/f923059d-f701-4b2d-9a40-8b254d2f6350) ou através do link: https://lookerstudio.google.com/s/m4rwlEFb50s
 * **Modelo de Processamento**: [Kaggle](https://www.kaggle.com/code/rafieb/analise-de-dados-covid-19-dashboard-rafieb).
 * **Fontes**: [Casos pela universidade John Hopkins](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports); [Vacinação pela universidade de Oxford](https://covid.ourworldindata.org/data/owid-covid-data.csv).
  
## Pandemia SARS-CoV-2

A COVID-19 é uma infecção respiratória aguda causada pelo coronavírus SARS-CoV-2, potencialmente grave, de elevada transmissibilidade e de distribuição global. Fonte: [Governo do Brasil](https://www.gov.br/saude/pt-br/coronavirus/o-que-e-o-coronavirus).

A disponibilidade de dados sobre a evolução dessa pandemia no tempo em uma determinada região geográfica é fundamental para entendimento e combate. Este caderno visa construir um dashboard para explorar o desenvolvimento da COVID-19 através de projeções de dados, exploração e visualização interativa sobre o avanço de casos e vacinação no Brasil. O dashboard encontra-se no [Looker Studio](https://lookerstudio.google.com/reporting/f923059d-f701-4b2d-9a40-8b254d2f6350). Enjoy!
    

* [Dados](#Dados)<br>
* [Análise de Dados Exploratória](#EDA)<br>
* [Dashboard Interativo](#Dashboard)<br>

## Dados
  

Os dados sobre **casos da COVID-19** são compilados pelo centro de ciência de sistemas e engenharia da [**John Hopkins University**](https://www.jhu.edu). Os dados são atualizados diariamente deste janeiro de 2020 com uma granularidade temporal de dias e geográfica de regiões de países (estados, condados, etc.).
Confira o projeto [JHU CCSE COVID-19 Content Portal](https://systems.jhu.edu/research/public-health/ncov/) e acesse os dados diários no [GitHub da JHU](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports). Abaixo estão descritos os dados derivados do seu processamento.

 - **date**: data de referência;
 - **state**: estado;
 - **country**: país;
 - **population**: população estimada;
 - **confirmed**: número acumulado de infectados;
 - **confirmed_1d**: número diário de infectados;
 - **confirmed_moving_avg_7d**: média móvel de 7 dias do número diário de infectados;
 - **confirmed_moving_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **deaths**: número acumulado de mortos;
 - **deaths_1d**: número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias do número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **month**: mês de referência;
 - **year**: ano de referência.

Os dados sobre **vacinação da COVID-19** são compilados pelo projeto Our World in Data (OWID) da [**University of Oxford**](https://www.ox.ac.uk). Os dados são atualizados diariamente deste janeiro de 2020 com uma granularidade temporal de dias e geográfica de países. Confira o projeto [OWID](https://ourworldindata.org) e baixe os [dados em CSV](https://covid.ourworldindata.org/data/owid-covid-data.csv). Abaixo estão descritos os dados derivados do seu processamento.
    
 - **date**: data de referência;
 - **country**: país;
 - **population**: população estimada;
 - **total**: número acumulado de doses administradas;
 - **one_shot**: número acumulado de pessoas com uma dose;
 - **one_shot_perc**: número acumulado relativo de pessoas com uma dose;
 - **two_shots**: número acumulado de pessoas com duas doses;
 - **two_shot_perc**: número acumulado relativo de pessoas com duas doses;
 - **three_shots**: número acumulado de pessoas com três doses;
 - **three_shot_perc**: número acumulado relativo de pessoas com três doses;
 - **month**: mês de referência;
 - **year**: ano de referência.

## Análise de Dados Exploratória

In [ ]:
# bibliotecas
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

### Casos
#### Extração dos casos

In [ ]:
# O dado está compilado em um arquivo por dia, exemplo para 2021/12/01.
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')
cases.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-13 05:22:15,33.93911,67.709953,53584,2301,44608,6675,Afghanistan,137.647787,4.294192
1,NaN,NaN,NaN,Albania,2021-01-13 05:22:15,41.15330,20.168300,64627,1252,38421,24954,Albania,2245.708527,1.937271
2,NaN,NaN,NaN,Algeria,2021-01-13 05:22:15,28.03390,1.659600,102641,2816,69608,30217,Algeria,234.067409,2.743543
3,NaN,NaN,NaN,Andorra,2021-01-13 05:22:15,42.50630,1.521800,8682,86,7930,666,Andorra,11236.653077,0.990555
4,NaN,NaN,NaN,Angola,2021-01-13 05:22:15,-11.20270,17.873900,18343,422,15512,2409,Angola,55.811022,2.300605


In [ ]:
# define iterador para um intervalo de tempo a ser extraído
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)
print("date function defined")

func defined


In [ ]:
# define o intervalo de tempo a ser extraído
start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)
print("defined date", start_date, end_date)

defined 2021-01-01 00:00:00 2021-12-31 00:00:00


In [ ]:
# define iterador para selecionar as colunas de interesse e as linhas referentes ao Brasil.
cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')

  case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
  case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

  if cases_is_empty:
    cases = case
    cases_is_empty = False
  else:
    cases = pd.concat([cases,case],ignore_index=True)
print("selection finished + dataframe ready")

selection finished


In [ ]:
# confere dados para 'Province_State == "Sao Paulo"'
cases.query('Province_State == "Sao Paulo"').head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
24,Sao Paulo,Brazil,1466191,46775,3192.990778,2021-01-01
51,Sao Paulo,Brazil,1467953,46808,3196.827966,2021-01-02
78,Sao Paulo,Brazil,1471422,46845,3204.382565,2021-01-03
105,Sao Paulo,Brazil,1473670,46888,3209.278136,2021-01-04
132,Sao Paulo,Brazil,1486551,47222,3237.329676,2021-01-05


#### Transformação dos casos

In [ ]:
cases.head()
cases.shape
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9828 entries, 0 to 9827
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province_State  9828 non-null   object        
 1   Country_Region  9828 non-null   object        
 2   Confirmed       9828 non-null   int64         
 3   Deaths          9828 non-null   int64         
 4   Incident_Rate   9828 non-null   float64       
 5   Date            9828 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 460.8+ KB


In [ ]:
# ajusta o nome das colunas
cases = cases.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)

for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})
print('columns adjusted')

columns adjusted


In [ ]:
# ajusta o nome dos estados.
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)
print('names adjusted')

names adjusted


In [ ]:
# organiza chaves temporais
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year']  = cases['date'].apply(lambda date: date.strftime('%Y'))
print('keys adjusted')

keys adjusted


In [ ]:
# extrai a população estimada a partir de 'incident_rate'
cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)
print('population extrated')

population extrated


In [ ]:
# casos e mortes por estado

cases_ = None
cases_is_empty = True

# define tendência como string a partir da estabilidade (14 dias)
def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.75:
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status

# Número
# média móvel (7 dias)
# estabilidade (14 dias)
# tendência

for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])
  # casos
  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)
  # mortes
  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    cases_ = pd.concat([cases_,cases_per_state], ignore_index=True)

cases = cases_
cases_ = None
print('parameters extracted')

parameters extracted


In [ ]:
# padroniza o tipo dos dados - type casting
cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')
print('type casting done')

type casting done


In [ ]:
# reorganizar as colunas
cases = cases[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]
cases.head(n=25)
print('columns adjusted')

columns adjusted


#### Carregamento dos casos

In [ ]:
# salva em './covid-cases.csv'
cases.to_csv('./covid-cases.csv', sep=',', index=False)
print('CSV dataframe saved')

CSV dataframe saved


### Vacinação
#### Extração de vacinações

In [ ]:
# Os dados estão compilados em um único arquivo.
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)
vaccines.head()
print("dataframe ready")

/tmp/ipykernel_47/2633929795.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)
/tmp/ipykernel_47/2633929795.py:2: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)


selection finished + dataframe ready


In [ ]:
# seleciona as colunas de interesse e as linhas referentes ao Brasil.
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]
vaccines.head()
print("selection finished")

selection finished


#### Transformação de vacinações

In [ ]:
vaccines.shape
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1428 entries, 0 to 1427
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   location                 1428 non-null   object        
 1   population               1428 non-null   float64       
 2   total_vaccinations       695 non-null    float64       
 3   people_vaccinated        691 non-null    float64       
 4   people_fully_vaccinated  675 non-null    float64       
 5   total_boosters           455 non-null    float64       
 6   date                     1428 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 78.2+ KB


In [ ]:
# preenche os dados faltantes com o valor anterior válido mais próximo.
vaccines = vaccines.fillna(method='ffill')
print('fillna ffill done')

fillna ffill done


In [ ]:
# filtra a base de dados para o mesmo período de "cases" pela coluna `date`.
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)
print('date selection done')

date selection done


In [ ]:
# ajusta o nome das colunas
vaccines = vaccines.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)
print('names adjusted')

names adjusted


In [ ]:
# organiza chaves temporais
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))
print('keys adjusted')

keys adjusted


In [ ]:
# relativização dos dados
vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)
print('parameters extrated')

parameters extrated


In [ ]:
# padroniza o tipo dos dados - type casting
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')
print('type casting done')

type casting done


In [ ]:
# reorganizar as colunas
vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]
vaccines.tail()
print('columns adjusted')

columns adjusted


#### Carregamento de vacinações

In [ ]:
# salva em './covid-vaccines.csv'
vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)
print('CSV dataframe saved')

CSV dataframe saved


## Dashboard Interativo
    
* #### **Dashboard Interativo**: [Looker Studio](https://lookerstudio.google.com/reporting/f923059d-f701-4b2d-9a40-8b254d2f6350) ou através do link https://lookerstudio.google.com/s/m4rwlEFb50s    

    
### **KPIs**
O dashboard de dados contem os seguintes indicadores chaves de desempenho (*key performance indicator* ou KPI) consolidados:
1. Casos e mortes nas 24 horas;
1. Média móvel (7 dias) de casos e mortes;
1. Tendência de casos e mortes;
1. Proporção de vacinados com 1ª, 2ª e 3ª doses.

### **EDA**
O dashboard de dados contem os seguintes gráficos para a análise exploratória de dados (*exploratory data analysis*
ou EDA) interativa:
1. Distribuição do números de casos e mortes ao longo do tempo;
1. Distribuição da média móvel (7 dias) do números de casos e mortes ao longo do tempo;
1. Distribuição geográfica dos casos por estado por dia.